##  Data Cleaning Techniques

The previous section was dedicated to identifying errors and quality issues in the data. Now we finally get to do something about it! In this section, we'll step through some basic approaches you can take to address the issues we identified in the web-scraped LinkedIn dataset. Growing your data preparation skills will be an ongoing effort throughout your career, so let's get started. 
  
The scenario for this section is a continuation of the work done for the previous section. To review, you have been asked by the head of recruiting for a tech company to analyze the data science jobs that are being advertised on LinkedIn to predict the most competitive salary bands and bonuses the company should be offering to employees and new hires to attract/retain talent. 
  
In this section, we are going to use pandas and matplotlib, so we’ll start by importing those libraries using aliases:

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

Let's import the LinkedIn dataset as `jobs`:

In [2]:
# read the CSV file into a pandas DataFrame 
jobs = pd.read_csv("../datasets/li-jobs-usa.csv")

# display the first five rows
jobs.head()

,title,company,description,onsite_remote,salary,sign_on_bonus,annual_bonus,location,criteria,posted_date
0,Data Analyst - Recent Graduate,paypal,"At PayPal (NASDAQ: PYPL), we believe that ever...",onsite,"$75,000.00\n -\n $95,000.00",9000,2400.0,Buffalo-Niagara Falls Area,"[{'Seniority level': 'Not Applicable'}, {'Empl...",NaN
1,Data Analyst - Recent Graduate,paypal,"At PayPal (NASDAQ: PYPL), we believe that ever...",onsite,NaN,4000,5400.0,"San Jose, CA","[{'Seniority level': 'Not Applicable'}, {'Empl...",NaN
2,Data Analyst,paypal,"At PayPal (NASDAQ: PYPL), we believe that ever...",onsite,NaN,2000,1200.0,"Texas, United States","[{'Seniority level': 'Not Applicable'}, {'Empl...",NaN
3,Data Analyst,PayPal,"At PayPal (NASDAQ: PYPL), we believe that ever...",onsite,NaN,3000,1800.0,"Illinois, United States","[{'Seniority level': 'Not Applicable'}, {'Empl...",NaN
4,Entry-Level Data Analyst,The Federal Savings Bank,"The Federal Savings Bank, a national bank and ...",onsite,NaN,4000,2400.0,"Chicago, IL","[{'Seniority level': 'Entry level'}, {'Employm...",NaN


### Part I. Missing Values

With variables with high percentages of missing values, we have several options. Some basic ones are: remove the variable, replace missing values with a reasonable value given the data or with a notation like "Unknown", or remove the observations (rows) which are missing data. The choice depends on how much data is missing and the importance of the variable for your analysis. 
 
Let's review the percentage of missing values from the columns in the `jobs` dataset:

In [ ]:
# display percentage of missing values for each column
jobs.isnull().mean()*100

<b>Case 1: `posted_date` </b>
Most of the observations in the `jobs` dataset are missing values for this variable, and it isn't important to our analysis, so we'll remove it using the `drop()` method: 

`df_name.drop(columns=[column 1, ...column n], inplace=[True|False])` 


The `columns` parameter takes a list of column names that will be removed from the DataFrame. The `inplace` parameter takes a boolean value specifying whether to change the current DataFrame in place or not. So, let's remove the `posted_date` column:  


In [ ]:
# drop column(s)
jobs.drop(columns=['posted_date'],inplace=True)

# display the first five rows
jobs.head()

<b>Case 2: `salary` </b>
The `salary` variable also has a high percentage of missing data, but will be very important to the predictive work we are prepping this data for. So instead of removing the entire column, we will only remove the rows which contain missing values using the `dropna()` method: 

`df_name.dropna(subset=[column 1, ...column n], inplace=[True|False])` 


The `subset` parameter takes a list of column names that let the `dropna()` method know which rows should be removed based on missing values on those columns. Let's remove the rows which contain missing values in the `salary` column:

In [ ]:
# remove rows with missing values in particular column
jobs.dropna(subset=['salary'], inplace=True)

# display the first five rows
jobs.head()

<b>Case 3: `annual_bonus` </b>

This variable is important to our analysis since it is about compensation, and it is missing less than 1% of its data. So, we will replace the missing values with reasonable estimates using imputation. Imputation means replacing the missing data with reasonable estimates given the data so that later data processing tasks can still work with the complete dataset. Commonly used reasonable estimates are: the average, median, or most frequent value of a column. 

The method to replace missing values in pandas is the `fillna()` method: 

`df_name["column_name"].fillna(replacement_value, inplace=[True|False])` 

We'll use the average value of the `annual_bonus` column to fill in the missing values: 

In [ ]:
# assign the average of a column into a variable
avg = jobs['annual_bonus'].mean()

# replace missing values with specified value
jobs['annual_bonus'].fillna(avg, inplace=True)

<b>Case 4: `location` </b>

Lastly, we have the `location` column that is not numerical and is missing values. Replacing the missing values with the mean is not an option for categorical variables. Instead, we'll replace the data with a placeholder value that can be filtered later. We'll use the word `Unknown` to replace the missing values:

In [ ]:
# replace missing values with replacement value
jobs['location'].fillna('Unknown', inplace=True)

### Part II. Outliers

Outliers can have more significant implications on the data than what we've previously discussed about missing values. For instance, an outlier might be an unexpected yet plausible data point, unquestionably part of the data set. Alternatively, an outlier might significantly deviate from the rest of the data, possibly indicating experimental errors or mistakes. Therefore, the cause and meaning of an outlier should be thoroughly investigated. Depending on the findings of this investigation, various actions can be taken.  

Let's revisit the box plot from the previous section, which depicts the outliers in the `sign_on_bonus` column:

In [ ]:
# create a boxplot 
plt.boxplot(jobs['sign_on_bonus'])

# annotations to help with identifying outliers
plt.annotate('outliers', xy=(.97, 9000), xytext=(.65, 8000), arrowprops=dict(facecolor='black', width=1))
plt.annotate('outliers', xy=(.97, 8000), xytext=(.65, 8000), arrowprops=dict(facecolor='black', width=1))

# modify the values of the ticks
plt.xticks([1], ['sign_on_bonus'])

# label y-axis
plt.ylabel("USD")

# display the figure 
plt.show()

The first thing and, above all, easiest thing to do would be to just leave the data as it is in the dataset! Since the `sign_on_bonus` column represents sign-on bonuses given to new employees for accepting an offer, high sign-on bonuses are a fact of reality that depends on the company, subject expertise, and/or years of experience, among a number of other factors.  

However, since there are only two outliers here, this means that only 0.072% of the data is an outlier. An incredibly small number! Thus, we are going to replace the outlier values with something more central to the distribution.  

The following code replaces both outlier values with the max value of the distribution without outliers by first finding the index values of the rows which contain the outliers and then directly assigning them the new max value:  

In [ ]:
# assign max value of distribution without the outliers
max_value = jobs[jobs['sign_on_bonus'] < 7000].sign_on_bonus.max()

# assign indices of rows which needs to be replaced (the rows with the outliers)
indices = jobs[jobs['sign_on_bonus'] > 7000].index

# replace the outlier values with replacement value
jobs.loc[indices,'sign_on_bonus'] = max_value

# verify that the values have been changed
jobs.loc[indices]

Note that the replacement value could have been something like the average, like the missing values section, but, in this case, the max is closer to the outliers' value than the average would be so using the max changes the dataset less than using the average would.   

### PI 5.3.4 Unnecessary

Unlike the prior two sections where missing values or outliers could be substituted, dealing with unnecessary data is more direct, essentially involving its removal from the dataset. It's crucial to remember that the only data that should remain in the dataset is that which is relevant and adds value.

To that end, we revisit the number of values found within the `onsite_remote` column:

In [ ]:
# output the frequency of value occurrences in column
jobs['onsite_remote'].value_counts()

Given the low variation in data, the utility of this data for training a predictive model greatly depends on the specific objective. Even though we can calculate the frequency of occurrences from data that isn't "balanced" or equally distributed, like the 'onsite_remote' column, a high-frequency value like 'onsite' could potentially introduce a bias into the model. 

To remove the unnecessary column, once again, we'll use the `drop()` method:

In [ ]:
# drop column(s)
jobs.drop(columns=['onsite_remote'], inplace=True)

##### Duplicate Rows

Remember that another problem we found with the LinkedIn dataset was duplicated rows. 

We don't need slicing or indexing to remove duplicate rows. Instead, pandas provides us with the handy `drop_duplicates()` method:

`df_name.drop_duplicates(inplace=[True|False])`

Let's remove the duplicates in our LinkedIn dataset:

In [ ]:
# output all the rows which are duplicated
jobs[jobs.duplicated()]

No slicing or indexing is needed to actively remove those duplicate rows as pandas provide the `drop_duplicates()` method. The `drop_duplicates()` method is used as follows:

`df_name.drop_duplicates(inplace=[True|False])`

Let's remove the duplicates in our LinkedIn dataset:

In [ ]:
# remove rows that are duplicated
jobs.drop_duplicates(inplace=True)

### PI 5.3.5 Inconsistent

The best course of action with unnecessary data is to simply remove it. On the other hand, when dealing with inconsistent data, the goal is to modify it in such a way that it becomes consistent. This ensures that the information contained within the inconsistent data remains unaltered following the manipulation.

Let's review the values for the `company` column in the LinkedIn dataset that have inconsistencies:

In [ ]:
# Add the casefolded version of the variable you want to check for inconsistencies to the DataFrame
jobs['company_casefolded'] = jobs['company'].str.casefold()

# Compare the original and the casefolded versions
jobs['Is_Inconsistent_company'] = jobs['company'] != jobs['company_casefolded']

# Identify the False values since we want company names to be capitalized.
jobs[~jobs["Is_Inconsistent_company"]]

Let's isolate which companies have lower case names:

In [ ]:
# output unique companies with lower case names
jobs[~jobs["Is_Inconsistent_company"]]["company"].unique()

We see that there are three of them. Now we need to decide which ones to replace. A little web searching reveals that PayPal and Alice + Olivia are capitalized, but fluent360 is not. So we'll replace all the lower case "paypal" and "alice + olivia" instances with "PayPal" and "Alice + Olivia", but leave "fluent360" alone using the `str.replace()` method like this: 

`df_name["column_name"] = df_name["column_name"].str.replace("value_to_replace", "replacement_value")`


Unlike some of the other replacement methods we've used, the `str.replace()` method does not include an `inplace` parameter. For this method, we apply the `str.replace()` method to a column and overwrite the old column with the new column with all the replacements. 


Since we need to replace more than one value in our LinkedIn dataset, we'll chain the methods together:

In [ ]:
# replace the column values to match preferred case
jobs['company'] = jobs['company'].str.replace('paypal','PayPal').str.replace( 'alice + olivia' ,'Alice + Olivia')

Now let's revisit the issues we identified with the `salary` column:

In [ ]:
# read the CSV file into a pandas DataFrame 
jobs = pd.read_csv("../datasets/li-jobs-usa.csv")

# display the first five rows
jobs.head()

In [ ]:
# display first five rows of value counts
jobs['salary'].value_counts().head()

Issues:

- The column name, "salary", might imply a single, yearly salary figure, but instead, this appears to be a salary range column.

- There are special characters like `\n` and lots of whitespace between the values which correspond to carriage returns and newlines (this could be due to the web-scraping method used).

- The widely varying values for salary indicate that some of the entries are giving salaries per hour while others are giving yearly figures.


#### So what do we do? 


We'll turn  `salary` into two separate numerical columns, `salary_min` and `salary_max,` with annual, not hourly, salary figures. 

In other words, we'll turn observations in our current salary column that look like this: 

`$50.00\n            -\n            $60.00` 

into two columns: 

`salary min: 100000` and `salary max: 120000`

This takes a few steps. 

First, we'll remove the characters in the `salary` column that are not consistent with a numerical representation of monetary values, namely `$` and `,`, using our familiar `str.replace` method:

In [ ]:
# replace characters with empty string in column values
jobs['salary'] = jobs['salary'].str.replace('$','',regex=False)
jobs['salary'] = jobs['salary'].str.replace(',','',regex=False)

The next step is to break apart the `salary` column into two columns. We do this we parse the string values representing the salary range by using the `str.split()` method like this:

`new_df_name = df_name[column_name].str.split("separator" , expand=[True|False])` 

If the `expand` parameter is `True`, then the return value is a DataFrame where each column is one of the partitioned strings parsed out according to the separator. 

Let's see what this looks like before we apply it to the `jobs` DataFrame:

In [ ]:
# split the strings in the column into a DataFrame
split_string_salaries = jobs['salary'].str.split('-', expand=True)
 
# display DataFrame
split_string_salaries

Now, we create two new columns in the `jobs` DataFrame based on the `split_string_salaries` DataFrame and drop the original `salary` column:

In [ ]:
# create new columns based on split string
jobs['salary_min'] = split_string_salaries[0]
jobs['salary_max'] = split_string_salaries[1]
 
# drop unnecessary column
jobs.drop(columns=['salary'], inplace=True)
 
# display first five rows
jobs.head()

Now we need to turn these new columns from strings into numbers so we can compute the annual salary based on those values per hour. We'll use the familiar `astype()` method:

In [ ]:
# change datatypes to float for computation
jobs['salary_min'] = jobs['salary_min'].astype(float)
jobs['salary_max'] = jobs['salary_max'].astype(float)

Finally, we want all our salary values as annual values. So, how do we know which ones they are? A quick way to visualize the divide between per-hour salaries and annual salaries is to use a histogram:

In [ ]:
# create histogram plot
plt.hist(jobs['salary_min'],bins=20)

# label x-axis
plt.xlabel('USD')

# label y-axis
plt.ylabel('Count')

# display figure
plt.show()

The large gap between 20,000 and 45,000 tells us that anything in the first bin is a salary per hour value. The next step is to create a filter to isolate all the rows with salary per hour values so that we can convert them. We'll select all `jobs['salary_min']`values less than 5000 USD:  


In [ ]:
# assign filter for data with hourly wages
hourly_wage_filter = (jobs['salary_min'] < 5000)
 
# get indices for rows with hourly wages
jobs_with_hourly_wage = jobs[hourly_wage_filter]
 
# verify our values before the change
jobs[hourly_wage_filter]

We'll use the `loc[]` method to grab the values and convert them to annual figures by multiplying the hourly wage by 40 x 52 (a 40-hour work week with 52 weeks in a year):


In [ ]:
# replace salary values with annual salary (hourly * 40 hours * 52 weeks) 
jobs.loc[hourly_wage_filter, 'salary_min'] = jobs_with_hourly_wage['salary_min'] * 2080
jobs.loc[hourly_wage_filter, 'salary_max'] = jobs_with_hourly_wage['salary_max'] * 2080

# verify that values have been changed
jobs[hourly_wage_filter]